# Convert Compability Matrix to make training data

In [1]:
import pandas as pd

df_compat = pd.read_excel('data/Option compat matrix 24 updated 29 May.xlsx')

df_compat = df_compat[['OPTION CODE', 'OPTION CODE COMPAT']]

print(f'Shape of dataframe with duplicates is {df_compat.shape}')

df_compat = df_compat.drop_duplicates()

print(f'Shape of dataframe without duplicates is {df_compat.shape}')

df_compat

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

## Remove Some Actions from the dataset (for testing)

In the intial experiment 

## Convert Action txt files into large JSON

In [8]:
import json
import glob
import unicodedata
import re

# Function to replace u/XXXX with the corresponding Unicode character
def replace_unicode_notation(match):
    unicode_code = match.group(1)
    return chr(int(unicode_code, 16))

glob.glob('/Users/adamfletcher/Documents/GitHub/defra_geojson/actions/*.txt')
action_json = {}

for filepath in glob.glob('/Users/adamfletcher/Documents/GitHub/defra_geojson/actions/*.txt'):
    code = filepath.split('/')[-1].split('-')[0].upper()
    with open(filepath, 'r') as f:
        text = f.read()
    cleaned_text = re.sub(r'u/([0-9A-Fa-f]{4})', replace_unicode_notation, text)
    action_json[code] = cleaned_text

    # print(code)
    # print(text)
    # print()

In [9]:
action_json

{'HEF5': 'This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024. You must read the SFI scheme information to understand the scheme rules and how to apply. 5 years £215 per hectare (ha) per year This action’s aim is that there’s a well-managed, intact grass sward growing over the historic or archaeological feature throughout the year, with minimal scrub cover and bare ground. The purpose of this is to: You can do this action on land located above and below the moorland line that’s: Total or part of the available area in a land parcel. This action is static. This means you must do it at the same location each year of this action’s duration. You must manage the area containing the historic or archaeological feature in a way that can reasonably be expected to achieve this action’s aim. This includes: You must not: Before you remove scrub or trees, you must get any relevant consents, such as: You must do this action from its start date, throughout each

In [12]:
import itertools

actionlist = [key for key in action_json.keys()]
actionlist_cartesian = itertools.product(*[actionlist, actionlist])

In [11]:
for i in actionlist_cartesian:
    print(i)

('HEF5', 'HEF5')
('HEF5', 'PRF2')
('HEF5', 'GRH11')
('HEF5', 'BFS3')
('HEF5', 'CIPM4')
('HEF5', 'CHRW3')
('HEF5', 'CIPM1')
('HEF5', 'OFC4')
('HEF5', 'SCR2')
('HEF5', 'BND1')
('HEF5', 'CSAM2')
('HEF5', 'OFC5')
('HEF5', 'SOH3')
('HEF5', 'CNUM2')
('HEF5', 'UPL5')
('HEF5', 'CIGL1')
('HEF5', 'UPL2')
('HEF5', 'SOH4')
('HEF5', 'HEF2')
('HEF5', 'OFC2')
('HEF5', 'AHW11')
('HEF5', 'OFM1')
('HEF5', 'CHRW2')
('HEF5', 'AHW5')
('HEF5', 'WBD7')
('HEF5', 'CNUM1')
('HEF5', 'HEF1')
('HEF5', 'WBD6')
('HEF5', 'BFS4')
('HEF5', 'AHW9')
('HEF5', 'CAHL3')
('HEF5', 'OFM5')
('HEF5', 'UPL7')
('HEF5', 'SCR1')
('HEF5', 'BFS1')
('HEF5', 'PRF4')
('HEF5', 'HEF8')
('HEF5', 'UPL4')
('HEF5', 'CMOR1')
('HEF5', 'OFA6')
('HEF5', 'CHRW1')
('HEF5', 'CIPM3')
('HEF5', 'OFM3')
('HEF5', 'SOH1')
('HEF5', 'OFM2')
('HEF5', 'CAHL4')
('HEF5', 'AHW4')
('HEF5', 'OFA3')
('HEF5', 'WBD4')
('HEF5', 'PRF3')
('HEF5', 'BFS6')
('HEF5', 'AHW6')
('HEF5', 'WBD8')
('HEF5', 'UPL6')
('HEF5', 'SPM2')
('HEF5', 'GRH10')
('HEF5', 'GRH1')
('HEF5', 'WBD1'

## Construct training Dataset

In [152]:
## for some odd reason you need to run the previous cell before this will work!

import os

dataset = []
full_filename = 'full_dataset.jsonl'
reduced_filename = 'training_dataset.jsonl'

# Check if the file exists
if os.path.exists(full_filename):
    # Delete the file
    os.remove(full_filename)
    print(f"{full_filename} has been deleted.")
else:
    print(f"{full_filename} does not exist.")

if os.path.exists(reduced_filename):
    # Delete the file
    os.remove(reduced_filename)
    print(f"{reduced_filename} has been deleted.")
else:
    print(f"{reduced_filename} does not exist.")


for i in actionlist_cartesian:
    try:
        if i[0] == i[1]:
            continue
        text1 = action_json[i[0]]
        text2 = action_json[i[1]]
        compatibility = len(df_compat.loc[(df_compat['OPTION CODE'] == i[0]) & (df_compat['OPTION CODE COMPAT'] == i[1])])
        full_line = {'text1': text1, 'text2': text2, 'code1': i[0], 'code2': i[1], 'label': compatibility}
        with open(full_filename, 'a') as f:
            f.write(json.dumps(full_line) + "\n")
        reduced_line = {'text1': text1, 'text2': text2, 'label': compatibility}
        with open(reduced_filename, 'a') as f:
            f.write(json.dumps(reduced_line) + "\n")
    except:
        continue


full_dataset.jsonl does not exist.
training_dataset.jsonl does not exist.


In [153]:
## Shuffle the dict so that we can train a model on it
import random

with open('training_dataset.jsonl', 'r') as file:
    lines = file.readlines()
random.shuffle(lines)
with open('shuffled_training_data.jsonl', 'w') as file:
    for line in lines:
        file.write(line)

In [154]:
total_rows = len(lines)
train_size = int(total_rows * 0.7)
eval_size = int(total_rows * 0.15)
test_size = int(total_rows * 0.15)

print(f'Train Size: {train_size}, Eval Size: {eval_size}, Test Size: {test_size}')

eval_start = train_size
eval_end = train_size + eval_size
test_start = eval_end

Train Size: 7070, Eval Size: 1515, Test Size: 1515


In [155]:
train_dataset = lines[:train_size]
eval_dataset = lines[eval_start: eval_end]
test_dataset = lines[test_start:]

with open('train_dataset.jsonl', 'w') as file:
    for line in train_dataset:
        file.write(line)

with open('eval_datatset.jsonl', 'w') as file:
    for line in eval_dataset:
        file.write(line)

with open('test_dataset.jsonl', 'w') as file:
    for line in test_dataset:
        file.write(line)

In [144]:
json.loads(test_dataset[0])['text1']

'This is an action in the Sustainable Farming Incentive (SFI) scheme: expanded offer for 2024. You must read the SFI scheme information to understand the scheme rules and how to apply. 3 years £739 per hectare (ha) per year This action’s aim is that there’s an established pollen and nectar flower mix which: The purpose of this is to: You can do this action on agricultural land located below the moorland line that’s: This is a ‘limited area’ action. The total eligible area you enter into any combination of one of more of the ‘limited area’ actions must not be more than 25% of the total agricultural area of your farm. Read section 1.3 ‘SFI actions with a limited area’ for more information. Part of the available area in a land parcel. If you’re establishing a new pollen and nectar flower mix block or strip, this action is static. This means you must do it at the same location each year of this action’s duration. If you’re maintaining an existing pollen and nectar flower mix block or strip

In [158]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="full_dataset.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

In [159]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text1', 'text2', 'code1', 'code2', 'label'],
        num_rows: 10100
    })
})

In [165]:
len(dataset['train']['label'])
sum(dataset['train']['label'])
pcent_compatible = sum(dataset['train']['label'])/len(dataset['train']['label'])*100

print(f"{sum(dataset['train']['label'])}/{len(dataset['train']['label'])} actions are compatible")
print(f"{pcent_compatible}%")
print(f'Saying Not Compatible 100% of the time is {100-pcent_compatible}% accurate')

1485/10100 actions are compatible
14.702970297029703%
Saying Not Compatible 100% of the time is 85.29702970297029% accurate
